K Means Optimisation


In [2]:
%pylab
%matplotlib inline

Using matplotlib backend: <object object at 0x0000026D514465B0>
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [3]:
import numpy 
import math
import random
import operator
from PIL import Image

In [78]:
def calculate_distance(a,b):

    a_positions = numpy.array(a[0])
    b_positions = numpy.array(b[0])
    a_rgb = numpy.array(a[1])
    b_rgb = numpy.array(b[1])
    distance_vector_positions = a_positions - b_positions
    distance_vector_rgb = a_rgb - b_rgb
    distance_vector = numpy.concatenate([distance_vector_positions, distance_vector_rgb])
    distance = numpy.sqrt(sum(distance_vector**2))
    return distance

def initiate_centroids(K,image):
    imagePixels = getPositions(image)
    initial_centers = list()
    for x in range(K):
        initial_centers.append(imagePixels[random.randint(0,len(imagePixels)-1)])
    return initial_centers

def findClosestCentroids(centroids, image):
    
    imagePixels = getPositions(image)
    assigned_centroid = []
    for p in imagePixels:
        distance = []
        for c in centroids:
            distance.append(calculate_distance(c, p))
        assigned_centroid.append(numpy.argmin(distance))
    return assigned_centroid

In [79]:
calculate_distance([(100, 23),(0,     3, 43)], [(21, 35), (2, 50, 3)])

100.98514742277698

In [83]:
image = Image.open("../data/Yeast_Cells.jpg",'r')
findClosestCentroids(initiate_centroids(2, image), image)

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [53]:
def getPositions(img):
    width, height = img.size
    i = 0
    j = 0
    positions = []
    for i in range(width):
        for j in range(height):
            coordinate = i, j
            positions.append([(i, j), img.getpixel(coordinate)])
    return positions

In [54]:
image = Image.open("../data/Yeast_Cells.jpg",'r')
getPositions(image)


[[(0, 0), (2, 2, 0)],
 [(0, 1), (2, 4, 0)],
 [(0, 2), (1, 6, 0)],
 [(0, 3), (1, 6, 0)],
 [(0, 4), (4, 6, 1)],
 [(0, 5), (2, 4, 0)],
 [(0, 6), (2, 4, 1)],
 [(0, 7), (2, 4, 1)],
 [(0, 8), (1, 3, 0)],
 [(0, 9), (3, 2, 0)],
 [(0, 10), (3, 2, 0)],
 [(0, 11), (2, 2, 0)],
 [(0, 12), (2, 2, 0)],
 [(0, 13), (0, 5, 0)],
 [(0, 14), (2, 4, 1)],
 [(0, 15), (2, 4, 3)],
 [(0, 16), (0, 5, 1)],
 [(0, 17), (2, 2, 0)],
 [(0, 18), (3, 2, 0)],
 [(0, 19), (3, 2, 0)],
 [(0, 20), (2, 2, 0)],
 [(0, 21), (0, 5, 1)],
 [(0, 22), (1, 6, 0)],
 [(0, 23), (1, 6, 0)],
 [(0, 24), (2, 5, 0)],
 [(0, 25), (2, 2, 0)],
 [(0, 26), (5, 1, 0)],
 [(0, 27), (3, 2, 0)],
 [(0, 28), (3, 2, 0)],
 [(0, 29), (1, 1, 0)],
 [(0, 30), (0, 2, 0)],
 [(0, 31), (1, 3, 0)],
 [(0, 32), (2, 4, 1)],
 [(0, 33), (2, 4, 1)],
 [(0, 34), (2, 4, 0)],
 [(0, 35), (2, 2, 0)],
 [(0, 36), (3, 2, 0)],
 [(0, 37), (5, 1, 0)],
 [(0, 38), (5, 1, 0)],
 [(0, 39), (3, 2, 0)],
 [(0, 40), (3, 4, 0)],
 [(0, 41), (3, 5, 0)],
 [(0, 42), (3, 5, 0)],
 [(0, 43), (2, 4, 1)]

Unify centroid list and responsibility / only create responsibility matrix in the format  x*k*d (x = number of pixels, k = number of centroids, d = dimensions) as numpy array or list

In [84]:
def responsibility(centroids, image):
    imagePixels = getPositions(image)

    x = len(imagePixels)

    k = len(centroids)

    d = len(centroids[0])

    matrix = numpy.zeros((x,k,d))

    for p in range(x):

        distance = []

        for c in centroids:

            distance.append(calculate_distance(imagePixels[p], c))
        
        matrix[p,numpy.argmin(distance)] = [1,1,1]
    return matrix
        


Cluster means using the new responsibility matrix

In [86]:
def clustermeans(responsibility, image):

    Pixelarray = numpy.array(getPositions(image))

    clusters = len(responsibility[1])
    
    new_centroids = []

    for k in range(clusters):
        new_centroids.append(list(numpy.average(responsibility[:,k] * Pixelarray, axis = 0)))
    return new_centroids

K Means V2

In [87]:
def k_means2(image, K, iterations):
    
    centroids = initiate_centroids(K, image)

    new_centroids = []
    
    i = 0

    while i < iterations:  # Maximum of 6 iterations

        resmat = responsibility(centroids,image) 

        new_centroids = clustermeans(resmat, image) # New Centroids

        if new_centroids == centroids:  # End algortihm, if centroids don't change
            break
        
        centroids = new_centroids

        i += 1
        
    final_assignment = findClosestCentroids(centroids, image)



    rgb_array = numpy.array([[255,255,255]]*len(final_assignment))
    for i in range(3):
        rgb_array[:,i] = rgb_array[:,i]*final_assignment/max(final_assignment)
    rgb_array
    rgb_list = []
    for i in range(len(rgb_array)):
        rgb_list.append(tuple(rgb_array[i]))
    image_out = Image.new(image.mode,image.size)
    image_out.putdata(rgb_list)
    image_out

    return image_out  # Assignment list can be used to create picture


In [88]:
image = Image.open("../data/Yeast_Cells.jpg",'r')


segmented_image = k_means2(image, 2, 2)
segmented_image.save('../data/test_out2.png')
segmented_image # still somewhat inconsistent

ValueError: could not broadcast input array from shape (3,) into shape (2,)